In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from skimage import io
from sklearn import svm
from sklearn.metrics import accuracy_score
import cv2
from skimage.feature import greycomatrix, greycoprops
from sklearn.ensemble import RandomForestClassifier

In [2]:
N_ANGLE_BINS = 40
BIN_SIZE = 360 // N_ANGLE_BINS
LEG_LENGTH = 25
np.seterr(divide = 'ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [3]:
%run ../Preprocessing/Read_Data.ipynb

In [4]:
def get_hinge_feature(image):
	image = cv2.medianBlur(image, 3)
	bw_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

	contours, _= cv2.findContours(
				bw_image, cv2.RETR_TREE, 
				cv2.CHAIN_APPROX_NONE
				)
	contours = sorted(contours, key=cv2.contourArea, reverse=True)[1:]
	
	hist = np.zeros((N_ANGLE_BINS, N_ANGLE_BINS))
	for cnt in contours:
		n_pixels = len(cnt)
		if n_pixels <= LEG_LENGTH:
			continue
		
		points = np.array([point[0] for point in cnt])
		xs, ys = points[:, 0], points[:, 1]
		point_1s = np.array([cnt[(i + LEG_LENGTH) % n_pixels][0] for i in range(n_pixels)])
		point_2s = np.array([cnt[(i - LEG_LENGTH) % n_pixels][0] for i in range(n_pixels)])
		x1s, y1s = point_1s[:, 0], point_1s[:, 1]
		x2s, y2s = point_2s[:, 0], point_2s[:, 1]
		
		phi_1s = np.degrees(np.arctan2(y1s - ys, x1s - xs) + np.pi)
		phi_2s = np.degrees(np.arctan2(y2s - ys, x2s - xs) + np.pi)
		
		indices = np.where(phi_2s > phi_1s)[0]
		
		for i in indices:
			phi1 = int(phi_1s[i] // BIN_SIZE) % N_ANGLE_BINS
			phi2 = int(phi_2s[i] // BIN_SIZE) % N_ANGLE_BINS
			hist[phi1, phi2] += 1
			
	normalised_hist = hist / np.sum(hist)
	feature_vector = normalised_hist[np.triu_indices_from(normalised_hist, k = 1)]
	
	return feature_vector


In [5]:
x_train = np.array([get_hinge_feature((img)) for img in tqdm(x_train)])
x_test = np.array([get_hinge_feature((img)) for img in tqdm(x_test)])

 14%|█▎        | 37/271 [00:06<00:40,  5.75it/s]

KeyboardInterrupt: 